### Imports 

In [ ]:
import warnings
warnings.simplefilter("ignore")

from sktime.forecasting.compose import RecursiveTimeSeriesRegressionForecaster
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.forecasting.naive import NaiveForecaster
from sklearn.ensemble import RandomForestRegressor
from sktime.forecasting.arima import AutoARIMA
from sktime.utils.plotting import plot_series
import pandas as pd
import numpy as np

### Leitura do dataset, formatação dele e multiplicação por escalar 

In [ ]:
df = pd.read_csv("../data/populacao_2020-2023.csv")
df = df.T
df = df.rename(columns={0:"populacao"})
df.index = pd.to_datetime(df.index)
df.index = df.index.to_period("M")
df.populacao = pd.to_numeric(df.populacao)
df.populacao *= 1000

In [ ]:
df.plot(figsize=(10,3))

## Separação treino e teste

In [ ]:
y = df
y_train, y_test = temporal_train_test_split(y)
fh = np.arange(len(y_test))+1
plot_series(y_train, y_test, labels=["y_train", "y_test"])

## Previsões com o método Forecaster 

In [ ]:
# Naive Forecaster com estratégia 'last'

naive_forecaster_last = NaiveForecaster(strategy="last")
naive_forecaster_last.fit(y_train)
y_last = naive_forecaster_last.predict(fh)

# Naive Forecaster com estratégia 'drift' e sp=12

naive_forecaster_drift = NaiveForecaster(strategy="drift",sp=12)
naive_forecaster_drift.fit(y_train)
y_drift = naive_forecaster_drift.predict(fh)

plot_series(y_train, y_test, y_last,y_drift, labels=["y_train", "y_test", "y_pred_last","y_drift"]);

## Previsões com ARIMA

In [ ]:
arima_forecasting = AutoARIMA(sp=12)
arima_forecasting.fit(y_train)

# Modelo pode demorar um pouco pois irá testar vários parâmetros.
y_arima = arima_forecasting.predict(fh)

plot_series(y_train, y_test, y_arima, labels=['y_train', 'y_test', 'y_arima'])

## Conclusão
O modelo ARIMA preveu tão corretamente que sobrepôs a linha de y_test. Logo, é o melhor modelo para ser usado no tópico de População.